In [1]:
import pandas as pd
import numpy as np
import allel
import moments.LD

### Preprocessing of 1000G VCF files
1. Apply strict mask
    - Some sites are not reliably mapped in hg38 and needs to be excluded.
    - These regions include: repeat regions, pseudogenes, centromeric, and telomeric regions.
2. Select only biallelic SNP sites
    - A VCF can contain mult-allelic SNPs and structure variants (indels, tandem repeats, etc)
    - The biallelic SNP sites are the easiest to work with, and most studies focus on them. 



This process can take quite a long time, I saved the processed VCFs under ~/projects/ctb-sgravel/data/30x1000G_biallelic_strict_masked <br>
The example preprocessing script is provided below and also in the directory's README.md.

In [ ]:
%%sh
### in bash, preprocess using bcftools
### replace variable path
chr22_vcf_path='/home/alouette/projects/ctb-sgravel/data/1000G/20201028_CCDG_14151_B01_GRM_WGS_2020-08-05_chr22.recalibrated_variants.vcf.gz'
strict_mask_1000G='/home/alouette/projects/ctb-sgravel/data/1000G/masks_and_annotations/20160622.allChr.mask.bed'
IntegerT=6
module load bcftools
bcftools --version | head -1
### -Oz output compressed VCF format
### -m2 -M2 -v to only view biallellic SNPs
### -R subset VCF from region bed file
### --threads parallelization currently only works for compression
bcftools view -Oz -m2 -M2 -v snps -R ${strict_mask_1000G} ${chr22_vcf_path} --threads ${IntegerT} -o "chr22.strict_masked.vcf.gz"

bcftools manual: http://samtools.github.io/bcftools/bcftools.html#expressions

### Optional: covert the VCF to zarr format for faster retrieval in scikit-allele
In sgkit, there might be better approaches. <br>
I also completed this step and stored the output in ~/projects/ctb-sgravel/data/30x1000G_biallelic_strict_masked/zarrFormat

### Population information retrieval

### Formatting the recombination map for pipeline
Several recombination maps exist for the human population. Each is computed using specific populations and with specific techniques.

### Minimal example pipeline: EDAR region in chr2:107,500,000-110,000,000
EDAR is an identified sweep in the EAS population.

In [ ]:
%%sh
chr2_vcf_path='/home/alouette/projects/ctb-sgravel/data/1000G/20201028_CCDG_14151_B01_GRM_WGS_2020-08-05_chr2.recalibrated_variants.vcf.gz'
strict_mask_1000G='/home/alouette/projects/ctb-sgravel/data/1000G/masks_and_annotations/20160622.allChr.mask.bed'
IntegerT=6
module load bcftools
### takes 6 min
bcftools view -Oz -m2 -M2 -v snps -r chr2:107500000-110000000 ${chr2_vcf_path} --threads ${IntegerT} -o "chr2.EDAR_region.vcf.gz"
bcftools index --threads ${IntegerT} "chr2.EDAR_region.vcf.gz"
### takes 3 min
bcftools view -Oz -R ${strict_mask_1000G} "chr2.EDAR_region.vcf.gz" --threads ${IntegerT} -o "chr2.EDAR_region.strict_mask.vcf.gz"